# MISP

In [ ]:
%pip install pymisp

In [ ]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
from dotenv import dotenv_values

In [ ]:
from surianalytics.connectors import getGitRoot, check_str_bool

In [ ]:
import os
import shutil

In [ ]:
config = dotenv_values(os.path.join(getGitRoot(), ".env") if shutil.which("git") is not None else os.path.join(os.path.expanduser("~"), ".env"))

In [ ]:
from pymisp import PyMISP

In [ ]:
misp = PyMISP(config["MISP_HOST"],
              config["MISP_TOKEN"],
              check_str_bool(config["MISP_TLS_VERIFY"]),
              debug=False)

In [ ]:
publish_timestamp="7d"

In [ ]:
import pandas as pd

In [ ]:
attributes = [
    "url",
    "domain",
    "ip-dst",
    "ip-dst|port"
]

In [ ]:
DF_ATTR = pd.DataFrame()
for attr in attributes:
    resp = pd.json_normalize(misp.search(controller='attributes', 
                                                    publish_timestamp=publish_timestamp, 
                                                    type_attribute=attr, 
                                                    category="Network activity", 
                                                    pythonify=False).get("Attribute"))
    DF_ATTR = pd.concat([DF_ATTR, resp], axis=0)
    print(attr, len(resp))

In [ ]:
DF_ATTR.head(3)

In [ ]:
DF_ATTR.groupby("type").agg({"value": "nunique",
                             "event_id": "nunique"})

In [ ]:
DF_ATTR.groupby("Event.info").agg({"type": ["unique", "nunique"],
                                   "value": ["unique", "nunique"]})